# Master MIND - IDLE

# Modèles récurrents

------------------------------------------------------------------------

**Préparation**

Pour simplifier, voici une implémentation pour charger le dataset:

-   Chargement des données depuis HuggingFace
-   Pré-traitement de base, et découpage des mots, récupération des plus
    fréquents
-   Encodage des mots par des entiers
-   Emballage dans un Dataset `torch`

In [2]:
from datasets import load_dataset
import re
from collections import Counter
from torch.utils.data import DataLoader, Dataset
import torch

dataset = load_dataset("imdb")

def tokenize(text):
    text = text.lower()
    text = re.sub(r'[^a-z0-9\s]', '', text)
    return text.split()

def build_vocab(dataset_iter, max_vocab_size=10000):
    counter = Counter()
    for item in dataset_iter:
        tokens = tokenize(item['text'])
        counter.update(tokens)
    
    # Mots les plus fréquents
    # Padding <pad>
    # Mots inconnus <unk>
    most_common = counter.most_common(max_vocab_size)
    vocab = {'<pad>': 0, '<unk>': 1}
    for word, _ in most_common:
        vocab[word] = len(vocab)
    return vocab

vocab = build_vocab(dataset['train'])

README.md: 0.00B [00:00, ?B/s]

plain_text/train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

plain_text/test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

plain_text/unsupervised-00000-of-00001.p(…):   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [3]:
vocab

{'<pad>': 0,
 '<unk>': 1,
 'the': 2,
 'and': 3,
 'a': 4,
 'of': 5,
 'to': 6,
 'is': 7,
 'in': 8,
 'it': 9,
 'i': 10,
 'this': 11,
 'that': 12,
 'br': 13,
 'was': 14,
 'as': 15,
 'for': 16,
 'with': 17,
 'movie': 18,
 'but': 19,
 'film': 20,
 'on': 21,
 'not': 22,
 'you': 23,
 'are': 24,
 'his': 25,
 'have': 26,
 'he': 27,
 'be': 28,
 'one': 29,
 'its': 30,
 'at': 31,
 'all': 32,
 'by': 33,
 'an': 34,
 'they': 35,
 'from': 36,
 'who': 37,
 'so': 38,
 'like': 39,
 'her': 40,
 'just': 41,
 'or': 42,
 'about': 43,
 'has': 44,
 'if': 45,
 'out': 46,
 'some': 47,
 'there': 48,
 'what': 49,
 'good': 50,
 'when': 51,
 'more': 52,
 'very': 53,
 'even': 54,
 'she': 55,
 'my': 56,
 'no': 57,
 'up': 58,
 'would': 59,
 'which': 60,
 'only': 61,
 'time': 62,
 'really': 63,
 'story': 64,
 'their': 65,
 'were': 66,
 'had': 67,
 'see': 68,
 'can': 69,
 'me': 70,
 'than': 71,
 'we': 72,
 'much': 73,
 'well': 74,
 'been': 75,
 'get': 76,
 'will': 77,
 'into': 78,
 'also': 79,
 'because': 80,
 'other': 81

In [4]:
def encode_and_pad(text, vocab, max_len=200):
    tokens = tokenize(text)
    encoded = [ vocab.get(token, vocab['<unk>']) for token in tokens ]

    if len(encoded) > max_len:
        encoded = encoded[:max_len]
    else:
        encoded += [vocab['<pad>']] * (max_len - len(encoded))

    return torch.tensor(encoded, dtype=torch.long)

class IMDBDataset(Dataset):
    def __init__(self, hf_dataset, vocab, max_len=200):
        self.dataset = hf_dataset
        self.vocab = vocab
        self.max_len = max_len

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        text = self.dataset[idx]['text']
        label = self.dataset[idx]['label']
        x = encode_and_pad(text, self.vocab, self.max_len)
        y = torch.tensor(label, dtype=torch.float)
        return x, y

train_dataset = IMDBDataset(dataset['train'], vocab)
test_dataset = IMDBDataset(dataset['test'], vocab)

batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Exercice 1 - *Architecture* *RNN*

L’objectif est de construire un modèle de classification de sentiments
basique.

## Question 1

Construire une architecture simple:

-   une couche d’embedding `nn.Embedding`, initialisée aléatoirement
    (donc à apprendre),
-   une couche récurrent `nn.RNN` dont on récupérera l’état caché pour
    le passer à la couche suivante,;
-   une couche linéaire pour la classification finale

In [ ]:
# embedding: mot -> representation, maj uniquement le vec quon utilise

## Question 2

Entraîner et évaluer cette architecture (en train, en test, en analysant
les courbes d’apprentissage).

## Question 3

Adapter le module PyTorch pour que le nombre de couche récurrentes
devienne un hyper-paramètre.

Expérimenter avec différentes profondeurs

**Note**: avec `nn.RNN` il suffit d’utiliser l’argument `num_layers`
pour empiler plusieurs couches.

# Exercice 2 - *Comparaison* *avec* *LSTM* *et* *GRU*

## Question 4

Adapter le module précédent pour utiliser une ou plusieurs couches LSTM.
Analyser les performances.

## Question 5

Même question avec GRU (une variante de LSTM).

# Exercice 3 - *Embeddings* *pré-entraînés*

Au lieu d’apprendre les représentations des mots à partir d’une
initialisation aléatoir, nous allons utiliser des vecteurs GloVe
pré-entraînés.

In [ ]:
import gensim.downloader
embeddings = gensim.downloader.load("glove-wiki-gigaword-100")

## Question 6

L’embedding est utilisable comme un dictionnaire

In [ ]:
embeddings["that"]

Pour l’utiliser dans une architecture PyTorch, on doit récupérer les
représentations de tous les mots du vocabulaire et construire un
tenseur.

Nous devons donc créer une matrice de taille
`(vocab_size, embedding_dim)` où chaque ligne correspond au vecteur de
représentation GloVe du mot (ou zéro si inconnu).

## Question 7

Dans les architectures précédentes, utiliser la matrice précédente comme
initialisation de la couche d’embedding.

On n’oubliera pas de figer les poids de cette couche pour ne pas les
modifier lors de l’apprentissage.

## Question 8

Comparer les performances avec les modèles précédents.

# Exercice 4 - *Observation* *du* *vanishing* *gradient*

Le but de cet exercice est de constater expérimentalement le phénomène
de disparition du gradient dans des architectures profondes et
d’analyser l’impacts des différentes couches récurentes (RNN simple,
LSTM et GRU).

La documentation de
[`nn.RNN`](https://docs.pytorch.org/docs/stable/generated/torch.nn.RNN.html)
nous apprend le nom des attributs du module qui stockent les paramètres.

> Variables:
>
> -   weight_ih_l\[k\] – the learnable input-hidden weights of the k-th
>     layer, of shape (hidden_size, input_size) for k = 0. Otherwise,
>     the shape is (hidden_size, num_directions \* hidden_size)
> -   weight_hh_l\[k\] – the learnable hidden-hidden weights of the k-th
>     layer, of shape (hidden_size, hidden_size)
> -   bias_ih_l\[k\] – the learnable input-hidden bias of the k-th
>     layer, of shape (hidden_size)
> -   bias_hh_l\[k\] – the learnable hidden-hidden bias of the k-th
>     layer, of shape (hidden_size)

## Question 9

Écrire une fonction qui analyse un module torch pour récupérer les
normes des différents gradients dans les couches successives.

## Question 10

Adapter la boucle d’entraînement pour extraire les gradients. Afficher
leur évolution en fonction de la couche et en fonction de l’epoch.

Faire varier la profondeur et analyser.

## Question 11

Même question pour des LSTM et des GRU.